In [3]:
import gradio as gr
import numpy as np
import pandas as pd
import pickle
import joblib
from pandas.core.frame import DataFrame
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

cn = ["a4", "x8", "秦"]
carNameCap = [1,2,3,4,5,6,7,8]
fuelCap = ["汽油", "柴油", "液化石油气", "天然气", "混合动力", "其他", "电动"]
carb = ["豪华轿车", "小型车", "厢型车", "大巴车", "敞篷车", "双开门汽车", "商务车", "搅拌车"]
gearbox = ["自动", "手动"]
fuelsys = ["电动", "燃油"]
unfixed_damage = ["是", "否"]
sell = ["个体", "非个体"]
offer = ["出售", "求购"]

# 归一化
def scale_minmax(col):
    return (col-col.min())/(col.max()-col.min())

def car_price(data):

    # 模型读取
    # xgb_model = pickle.load(open('xgb.pkl', 'rb'))
    # lgb_model = pickle.load(open('lgb.pkl', 'rb'))
    xgb_model = joblib.load('xgb.pkl')
    lgb_model = joblib.load('lgb.pkl')
    print(xgb_model.feature_importances_)
    MAE_LGB = 624.05661
    MAE_XGB = 597.72837

    # 这部分照搬的处理流程,可能有问题
    # data = [car, fueltype, bodytype, kilomiter, usetype]
    #print(xgb_model.feature_names_in_)
    print(data)
    test_data_process = DataFrame(data,columns = xgb_model.feature_names_in_)
    #test_data_process1 = test_data_process.drop(labels=['SaleID','name','seller'],axis=1)
    #test_data_process.columns=['SaleID','name','regDate','model','brand','bodyType','fuelType','gearbox','power','kilometer','regionCode','seller','offerType','createDate','v_0','v_1','v_2','v_3','v_4','v_5','v_6','v_7','v_8','v_9','v_10','v_11','v_12','v_13','v_14']
    #test_data_process = test_data_process.apply(scale_minmax, axis=0)
    #test_data_process1 = test_data_process1.apply(scale_minmax, axis=0)
    
    '''
    pca = PCA(n_components=1)
    new_test_pca_4 = pca.fit_transform(test_data_process)
    new_test_pca_4 = pd.DataFrame(new_test_pca_4)
    '''
    
    out_lgb = lgb_model.predict(test_data_process)
    out_xgb = xgb_model.predict(test_data_process)
    print('lgb_out',out_lgb)
    print('xgb_out',out_xgb)
    # 结果输出
    final_out = (1 - MAE_LGB / (MAE_XGB + MAE_LGB)) * out_lgb + (1 - MAE_XGB / (MAE_XGB + MAE_LGB)) * out_xgb

    return final_out


def transform(carname, cartype, date_time_input, bodytype, fueltype, usetype, power, kilomiter, damage, region_cod, seller, offerType):
    carname = cn.index(carname)
    cartype = carNameCap.index(cartype)
    date_time_input = int(date_time_input)
    bodytype = carb.index(bodytype)
    fueltype = fuelCap.index(fueltype)
    usetype = gearbox.index(usetype)
    damage = unfixed_damage.index(damage)
    seller = sell.index(seller)
    offerType = offer.index(offerType)
    data = {'SaleID':[1],'name':[68349.172873],'regDate':[date_time_input],'model':[47.129021],'brand':[int(cartype)],  'bodyType':[bodytype],'fuelType':[fueltype],'gearbox':[usetype],'power':[power], 'kilometer':[kilomiter] ,     'regionCode':[region_cod],'seller':[0],'offerType':[offerType],'creatDate':[int(20160328)],'v_0':[44.406268],'v_1':[-0.044809],'v_2':[0.080765],'v_3':[0.078833],'v_4':[0.017875],'v_5':[0.248204],'v_6':[0.044923],'v_7':[0.124692],'v_8':[0.058144],'v_9':[0.061996],'v_10':[-0.001000],'v_11':[0.009035],'v_12':[0.004813],'v_13':[0.000313],'v_14':[-0.000688]}
    return car_price(data)
#data = {date_time_input,47.129021,int(cartype),  bodytype,fueltype,usetype,power, kilomiter ,     region_cod,offerType,20160328,44.406268,-0.044809,0.080765,0.078833,0.017875,0.248204,0.044923,0.124692,0.058144,0.061996,-0.001000,0.009035,0.004813,0.000313,-0.000688}
    
# 界面内要输入的东西
carname = gr.Radio(label="汽车名称", choices=cn)
cartype = gr.Radio(label="汽车品牌", choices=carNameCap)
date_time_input = gr.Textbox(label="购车时间（例如：20200114）")
bodytype = gr.Radio(label="汽车类别", choices=carb)
fueltype = gr.Radio(label="燃油种类", choices=fuelCap)
usetype = gr.Radio(label="变速箱", choices=gearbox)
power = gr.Slider(label="发动机功率", minimum=0, maximum=600)
kilomiter = gr.Slider(label="行驶里程(万公里)", minimum=0, maximum=15)
damage = gr.Radio(label="车身是否有尚未修复损坏", choices=unfixed_damage)
region_cod = gr.Radio(label="地区编码", choices=[10, 20, 30])
seller = gr.Radio(label="销售方", choices=["个体", "非个体"])
offerType = gr.Radio(label="买卖方", choices=["出售", "求购"])
output = gr.Textbox()

prediction = gr.Interface(title="二手车价格预测",
                       fn=transform,
                       inputs=[carname,
                               cartype,
                               date_time_input,
                               bodytype,
                               fueltype,
                               usetype,
                               power,
                               kilomiter,
                               damage,
                               region_cod,
                               seller,
                               offerType],
                       outputs=output)
# 展示
prediction.launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
